In [1]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection- Kaggle API

In [2]:
# kaggle_dictionary = json.load(open("kaggle.json"))

In [3]:
# kaggle_dictionary.keys()

In [4]:
# # setup kaggle credentials as environment variables
# os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
# os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [5]:
# !kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [6]:
# ls

In [7]:
# # unzip the dataset file
# with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
#   zip_ref.extractall()

Loading the Dataset

In [8]:
df = pd.read_csv("IMDB Dataset.csv")
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [9]:
df.shape

(50000, 2)

In [10]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [11]:
df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

C:\Users\pranav\AppData\Local\Temp\ipykernel_22660\1137712857.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [12]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [13]:
# split data into training data and test data
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [14]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Preprocessing

In [15]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [16]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [17]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [18]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [19]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


LSTM - Long Short-Term Memory

The root cause of this problem is that Keras sometimes delays "building" the layers of a model until it receives actual input data with a specified shape. This is known as `deferred model building`

In [20]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))  # Make sure input_length matches the sequence length
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Display the model summary
model.build(input_shape=(None, 200))#   # Specify input shape here to force layer building
model.summary()


c:\Users\pranav\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

Training the Model

In [22]:
model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 106s 207ms/step - accuracy: 0.7168 - loss: 0.5346 - val_accuracy: 0.8444 - val_loss: 0.3651
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 131s 261ms/step - accuracy: 0.8536 - loss: 0.3529 - val_accuracy: 0.8338 - val_loss: 0.3840
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 126s 253ms/step - accuracy: 0.8747 - loss: 0.3192 - val_accuracy: 0.8530 - val_loss: 0.3567
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 130s 260ms/step - accuracy: 0.8881 - loss: 0.2833 - val_accuracy: 0.8202 - val_loss: 0.4057
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 122s 244ms/step - accuracy: 0.8921 - loss: 0.2685 - val_accuracy: 0.8665 - val_loss: 0.3559
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 164s 328ms/step - accuracy: 0.8993 - loss: 0.2528 - val_accuracy: 0.8654 - val_loss: 0.3534
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 191s 383ms/step - accuracy: 0.9216 - loss: 0.2014 - val_accuracy: 0.8576 - val_loss: 0.3656
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 284ms/step - accuracy: 0.9308 -

Model Evaluation

In [23]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 36ms/step - accuracy: 0.8693 - loss: 0.3802
Test Loss: 0.37021782994270325
Test Accuracy: 0.8743000030517578


Building a Predictive System

In [ ]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [ ]:
# Example usage
new_review = "This movie was fantastic. I loved it."
print(f"RNN Prediction: {predict_sentiment_rnn(new_review)}")